# Chisel

![chisel_logo](pics/chisel_logo.svg) ![firrtl_logo](pics/firrtl_logo.svg) 

Chisel, как представляют его создатели - язык описаня цифровой апаратуры, который облегчает создание продвинутых генерируемых компонентов и их повторное использование при разработке rtl для asic и fpga. Chisel - scala DSL, по сути же является библиотекой scala с api в виде классов и singleton объектов с вышеописанными целями.

#### `*.scala (chisel)-> chirrtl (firrtl)-> firrtlhight, mid, low (firrtl)-> .v`

## Типы данных

![chisel_logo](pics/type_hierarchy.svg)

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.experimental._

### Базовые типы

In [ ]:
1.U
0xF.U

"hAAA".U
"o12".U
"b1010".U

5.S
-8.S

5.U

8.U(4.W)

-152.S(32.W)

true.B
false.B

22.7.F(16.BP)

### Аггрегатные типы

`Vec` и `Bundle` - сами по себе не являются полноценными типами данных, но позволяют пользователю объединять несколько типов.

In [ ]:
val vec = Vec(3, UInt(32.W))

In [ ]:
class MyBundle extends Bundle {
    val uint = UInt(32.W)
    val bool = Bool()
}
val bundle = new MyBundle
bundle.uint
bundle.bool

In [ ]:
class NestedBundle extends Bundle {
  val bundle = new MyBundle
  val vec = Vec(3, UInt(32.W))
    
  val bool = Bool()
}

val nested = new NestedBundle

nested.bundle.uint
nested.bundle.bool
nested.vec
nested.bool

In [ ]:
val mixed = util.MixedVec(UInt(3.W), UInt(10.W), Bool())

mixed(0)
mixed(1)
mixed(2)

** Используйте `Vec` только для портов или при необходимости динамического hardware доступа к массиву, для остального лучше использовать коллекции скала.

## Hardware wires, regs

Чизел четко, но не всегда явно разделят типы данных на абстрактные и hardware. Абстрактные типы служат для определения hardware сущностей.

![hardwareTypes](pics/hardwareType.svg)

In [ ]:
val dataType1 = UInt(32.W)
val dataType2 = UInt(32.W)

In [ ]:
val wire = Wire(UInt(32.W))

In [ ]:
val reg = Reg(UInt(32.W))

In [ ]:
val operation = dataType1 + dataType2

Нельзя создавать hardware объекты вне модуля. Типы данных сами по себе являются классами, но без обертки в hardware не будут влиять на verilog. Все hardware объекты являются объектами того же класса с помощью которых были созданы, магия происходит в коробке.

## Module, IO порты

Входной точкой chisel компилятора как правило служит метод объекта

`Driver.execute(args: Array[String], dut: () => RawModule)`

`args` - параметры компиляции, можно посмотреть полный список параметров тут https://github.com/freechipsproject/chisel3/wiki/Running-Stuff#what-are-all-the-options

часто используемый `--target-dir` задает путь для выходных `.fir` и `.v` файлов

`dut` - функция которая должна вернуть модуль отнаследованый как минимум от класса `RawModule`

Существует несколько типов модулей в chisel3 от которых можно наследоваться для имплементации модулей

- `chisel3.Module` он же `chisel3.internal.LegacyModule` - все порты должны быть описаны внутри io, имеет встроенный `clock` и `reset`, также имеется особености при использовании
- `chisel3.MultiIOModule` - порты не ограничены одним io, имеет встроенный `clock` и `reset`
- `chisel3.RawModule` - порты не ограничены одним io, НЕ имеет встроенного `clock` и `reset`
- `chisel3.BalckBox` - блэкбокс, об этом чуть позже

Рекомендация - использовать либо `MultiIOModule` или `RawModule`, рано или поздно `LegacyModule` должен перейти в статус deprecated

### IO

- Каждый порт внутри модуля оборачивается в `IO()`
- Все что обарачивается в `IO()` должно иметь направление `Input()`, `Output()` или `Analog()`
- `Bundle` может быть портом - наиболее удобный и частоиспользуемый вариант, в этом случае у каждого элемента должно быть указано направление или целиком у всего `Bundle`


### Мы готовы написать наш первый модуль (почти)

In [ ]:
class Invert32 extends RawModule {
  val in = IO(
    Input(UInt(32.W))
  )

  val out = IO(
    Output(UInt(32.W))
  )

  out := ~in
}

verilogPrint(new Invert32)
visualize(() => new Invert32)

## Комбинационная и не очень логика

Тут все просто и очень похоже на концструкции verilog

### Встроенные операции

их полный список можно найти тут https://github.com/freechipsproject/chisel3/wiki/Builtin-Operators

In [ ]:
class CombLogic extends RawModule {
  val io = IO(new Bundle {
    val a = Input(UInt(4.W))
    val b = Input(UInt(4.W))

    val out = Output(UInt(4.W))
  })

  io.out := ((io.a & io.b) | (io.a ^ ~io.b))  
}

verilogPrint(new CombLogic)
visualize(() => new CombLogic)

In [ ]:
class CombLogicIf extends RawModule {
  val io = IO(new Bundle {
    val ready = Input(Bool())

    val in = Input(UInt(32.W))

    val out = Output(UInt(32.W))
  })

  when(io.ready) {
    io.out := io.in + 255.U
  }.otherwise {
    io.out := io.in
  }
}

verilogPrint(new CombLogicIf)
visualize(() => new CombLogicIf)

### Mux

При описании комбинационной логики вместо `when(){}.elsewhen(){}.otherwise` рекомендую использовать встроенную функциональную абстрацкию `Mux()` и его варианты `MuxCase` и `MuxLookUp`(о них чуть позже), таким образом можно четко разделять комбинационную и последовательную логику в вашем дизайне.

In [ ]:
class CombLogicIfWihtMux extends RawModule {
  val io = IO(new Bundle {
    val ready = Input(Bool())
    
    val in = Input(UInt(32.W))
    
    val out = Output(UInt(32.W))
  })

  io.out := Mux(io.ready, io.in + 255.U, io.in)
}

verilogPrint(new CombLogicIfWihtMux)
visualize(() => new CombLogicIfWihtMux)

### Регистры

#### Базовая версия

In [ ]:
class Counter extends MultiIOModule {
  val io = IO(new Bundle {
    val enable = Input(Bool())
    val out = Output(UInt(32.W))
  })

  val nextCount = Wire(UInt(32.W))

  val counter = Reg(UInt(32.W))

  nextCount := Mux(counter === 100.U, 0.U, counter + 1.U)

  io.out := counter

  when(io.enable) {
    counter := nextCount
  }
}

verilogPrint(new Counter)
visualize(() => new Counter)

#### `RegInit`, `RegEnable`, `RegNext`

Разновидности конструкторов регистров для более удобной записи

In [ ]:
class CounterInit extends MultiIOModule {
  val io = IO(new Bundle {
    val enable = Input(Bool())
    val out = Output(UInt(32.W))
  })

  val nextCount = Wire(UInt(32.W))

  val counter = RegInit(0.U(32.W))

  nextCount := Mux(counter === 100.U, 0.U, counter + 1.U)

  io.out := counter

  when(io.enable) {
    counter := nextCount
  }
}

verilogPrint(new CounterInit)
visualize(() => new CounterInit)

In [ ]:
class CounterEnable extends MultiIOModule {
  val io = IO(new Bundle {
    val enable = Input(Bool())
    val out = Output(UInt(32.W))
  })

  val nextCount = Wire(UInt(32.W))

  val counter: UInt = RegEnable(
    next = nextCount,
    init = 0.U(32.W),
    enable = io.enable
  )

  nextCount := Mux(counter === 100.U, 0.U, counter + 1.U)

  io.out := counter
}

verilogPrint(new CounterEnable)
visualize(() => new CounterEnable)

## Кастинг
Или преобразование типов, 
```scala
asBool  : Bool
asBools : Seq[Bool]
asUInt  : UInt
asTypeOf: T
```

Может иногда являтся источником багов в вашем коде, так что при дебаге стоит обращать внимание на кастинг

In [ ]:
class SomeBundle extends Bundle {
  val uint = UInt(4.W)
  val bool1 = Bool()
  val bool2 = Bool()
  val bool3 = Bool()
  val bool4 = Bool()
}

class MultiIO extends RawModule {
  val io = IO(new Bundle {
    val in_bool = Input(Bool())
    val in_uint = Input(UInt(8.W))
    val in_vec = Input(Vec(4, UInt(2.W)))
    val in_bundle = Input(new SomeBundle)

    val out_bool = Output(Bool())
    val out_uint = Output(UInt(8.W))
    val out_vec = Output(Vec(4, UInt(2.W)))
    val out_bundle = Output(new SomeBundle)
  })

  io.out_bool := io.in_bool
  io.out_uint := io.in_vec.asUInt()
  io.out_vec := io.in_bundle.asTypeOf(Vec(4, UInt(2.W)))
  io.out_bundle := io.in_uint.asTypeOf(new SomeBundle)
}

verilogPrint(new MultiIO)
// visualize(() => new MultiIO)

## Интерфейсы и их соединение

Как можно догадаться `Bundle` может и должен быть использован для описания интерфейсов и их дальнейшего использования.
В этом случае необходимо указывать направление сигналов. Оператор `<>` позволяет соединять интерфейсы, для инверсии направлений сигналов используется метод `Flipped()`

In [ ]:
class ReadyValid extends Bundle {
  val ready = Input(Bool())
  val valid = Output(Bool())
  val bits = Output(UInt(32.W))
}

class ReadyValidMux extends RawModule {
  val io = IO(new Bundle {
    val addr = Input(UInt(8.W))

    val in = Vec(3, Flipped(new ReadyValid))
    val out = new ReadyValid
  })

  io.in.foreach(x => x.ready := false.B)

  io.out <> io.in.do_apply(io.addr.asUInt)
}

verilogPrint(new ReadyValidMux)
// visualize(() => new ReadyValidMux)

## Инстантирование модулей

In [ ]:
class Mux2 extends Module {
  val io = IO(new Bundle{
    val sel = Input(UInt(1.W))
    val in0 = Input(UInt(1.W))
    val in1 = Input(UInt(1.W))
    val out = Output(UInt(1.W))
  })
  io.out := (io.sel & io.in1) | (~io.sel & io.in0)
}

class Mux4 extends Module {
  val io = IO(new Bundle {
    val in0 = Input(UInt(1.W))
    val in1 = Input(UInt(1.W))
    val in2 = Input(UInt(1.W))
    val in3 = Input(UInt(1.W))
    val sel = Input(UInt(2.W))
    val out = Output(UInt(1.W))
  })
  val m0 = Module(new Mux2)
  m0.io.sel := io.sel(0) 
  m0.io.in0 := io.in0
  m0.io.in1 := io.in1

  val m1 = Module(new Mux2)
  m1.io.sel := io.sel(0) 
  m1.io.in0 := io.in2
  m1.io.in1 := io.in3

  val m3 = Module(new Mux2)
  m3.io.sel := io.sel(1) 
  m3.io.in0 := m0.io.out
  m3.io.in1 := m1.io.out

  io.out := m3.io.out
}

// verilogPrint(new Mux4)
visualize(() => new Mux4)


## Мощь - функциональная абстрацкия и глобальная параметризация

Т.к. chisel - это все еще scala, мы получаем все ее приемущества и возможности в виде параметризации, наследования и т.д.

### Функциональный код

In [ ]:
class GrayCounter extends MultiIOModule {
  val io = IO(new Bundle {
    val enable = Input(Bool())
    val out = Output(UInt(32.W))
  })

  println("Hello from counter!")

  val nextCount = Wire(UInt(32.W))

  val counter = RegEnable(
    next = nextCount,
    init = 0.U(32.W),
    enable = io.enable
  )

  nextCount := counter + 1.U

  io.out := utils.toGray(counter)
}

object utils {
  def toGray(in: UInt) = in ^ (in >> 1.U).asUInt
}

verilogPrint(new GrayCounter)

In [ ]:
object utils {
  def counter(max: Int, enable: Bool) = {
    val width = util.log2Ceil(max + 1)

    val nextCount = Wire(UInt(width.W))

    val count = util.RegEnable(
        nextCount,
        0.U(width.W),
        enable
    )

    nextCount := Mux(count === max.U, 0.U, count + 1.U)

    count
  }

  def toGray(in: UInt) = in ^ (in >> 1.U).asUInt
}

class CounterDoubleChannel extends MultiIOModule {
  val io = IO(new Bundle {
    val enable_1 = Input(Bool())
    val enable_2 = Input(Bool())

    val out_1 = Output(UInt(32.W))
    val out_2 = Output(UInt(64.W))
  })

  val counter1 = utils.counter(255, io.enable_1)
  val counter2 = utils.counter(2048, io.enable_2)

  io.out_1 := counter1
  io.out_2 := counter2
}

verilogPrint(new CounterDoubleChannel)

### Параметризация

#### Параметры конструктора модуля

In [ ]:
class ParameterizedCounter(max: Int) extends MultiIOModule {
  val io = IO(new Bundle {
    val enable = Input(Bool())
    val out = Output(UInt(32.W))
  })

  val nextCount = Wire(UInt(32.W))

  val counter = util.RegEnable(
    next = nextCount,
    init = 0.U(32.W),
    enable = io.enable
  )

  nextCount := Mux(counter === max.U, 0.U, counter + 1.U)

  io.out := counter
}

verilogPrint(new ParameterizedCounter(128))


In [ ]:
class MultiChannelCounter(maxValues: Seq[Int]) extends MultiIOModule {
  val io = IO(new Bundle {
    val enable = Input(Vec(maxValues.size, Bool()))

    // Vec(Seq(UInt(4.W), UInt(4.W), UInt(4.W)))
    val out = Output(MixedVec(maxValues.map { x => 
      UInt(log2Ceil(x + 1).W)
    }))
  })

  val counters = maxValues.zip(
    io.enable
  ).map { case(max, enable) =>
    utils.counter(max, enable)
  }

  counters.zip(
    io.out
  ).foreach { case(count, out) =>
    out := count
  }
}

verilogPrint(new MultiChannelCounter(Seq(16, 32, 64, 128)))

object MultiChannelCounter {
  def apply(max: Int *) = new MultiChannelCounter(max)
}

verilogPrint(MultiChannelCounter(16, 32, 64, 128))


#### Параметризация конструктора интерфейса


In [ ]:
class ValidIO(val _width: Int) extends Bundle {
  val valid = Output(Bool())
  val bits = Output(UInt(_width.W))

//   override def cloneType: this.type = { (new ValidIO(width)).asInstanceOf[this.type] }
}

#### Наследование

In [ ]:
class PipeLine(n: Int, width: Int) extends MultiIOModule {
  val io = IO(new Bundle {
    val in = Flipped(new ValidIO(width))
    val out = new ValidIO(width)
  })

  val stages = (1 to n).map(i => Reg(new ValidIO(width)))

  stages.zipWithIndex.foreach { case(stage, index) =>
    if(index == 0)
      stage := io.in
    else
      stage := stages(index - 1)
  }

  io.out := stages.last
}

visualize(() => new PipeLine(4, 32))


In [ ]:
class CountedPipeLine(n: Int, width: Int) extends PipeLine(n, width) {
  val count = IO(Output(UInt(log2Ceil(n + 1).W)))

  val counter = RegInit(0.U(log2Ceil(n + 1).W))

  count := counter

  counter := counter + io.in.valid - io.out.valid
}

visualize(() => new CountedPipeLine(4, 32))

#### Ремарка

Scala - язык со множеством возможностей, поэтому на уровне функциональной абстракции пользователю chisel доступен общирный набор средств и библиотек, вопрос как его применять решает исключительно сам разработчик и ограничен он лишь полетом фантазии.

Несколько советов : 

- Улучшение навыков и понимания языка scala полезно в любом случае, учиться учиться и еще раз учиться
- Не усложняйте в параметризации, запасы на все случае жизни - это хорошо но до добра не доведет
- Читайте исходники, всегда можно подчерпнуть интересные техники приемы и решения
- Практикуйтесь

## `util` - библиотека

### Функциональные блоки

`Reverse(in: UInt)` - разворачивает последовательность бит `UInt`

`Cat()` - операция конкатинации, аналог `{}` из verilog

`Counter(n: Int)` - счетчик

`MuxCase, MuxLookUp` - Более продвинутые аналоги `Mux`

`log2Ceil, log2Floor` - логарифмы по основанию 2 с различным типом округления, НЕ hardware элемент

`LFSR` - линейный регистр сдвига c обратной связью c конфигурацией Фибоначи, есть версия с конфигурацией Галуа, генерирует псевдослучайные значения заданной размерности 

### Интерфейсы

`ValidIO` - `bits` и `valid`

`DecoupledIO` - `bits`, `valid` и `ready`

### Модули

`Pipe` - pipeline модуль задержки c `ValidIO` интерфейсом

`Queue` - FIFO с `DecoupledIO` интерфейсами

`Arbiter` - арбитр на DecoupledIO интерфейсах


## Analog - inout порт

In [ ]:
class AnalogModule extends RawModule {
  val pad = IO(new Bundle {
    val a = Analog(1.W)
    val b = Analog(1.W)
    val c = Analog(1.W)
  })

  attach(pad.a, pad.b)
  attach(pad.a, pad.c)
}

verilogPrint(new AnalogModule)
visualize(() => new AnalogModule)

## Асинхронный сброс

In [ ]:
class AsyncResetCounter extends RawModule {
  val io = IO(new Bundle {
    val clock = Input(Clock())
    val reset = Input(AsyncReset())

    val count = Output(UInt(32.W))
  })

  withClockAndReset(io.clock, io.reset) {
    val counter = RegInit(0.U(32.W))

    counter := counter + 1.U

    io.count := counter
  }
}

verilogPrint(new AsyncResetCounter)
// visualize(() => new AsyncResetCounter)

### Multi clock domain


In [ ]:
class MultiClockCounter extends RawModule {
  val io = IO(new Bundle {
    val clock_one = Input(Clock())
    val reset_one = Input(AsyncReset())

    val clock_two = Input(Clock())
    val reset_two = Input(AsyncReset())

    val count_one = Output(UInt(32.W))
    val count_two = Output(UInt(32.W))

  })

  withClockAndReset(io.clock_one, io.reset_one) {
    val counter = RegInit(0.U(32.W))

    counter := counter + 1.U

    io.count_one := counter
  }

  withClockAndReset(io.clock_two, io.reset_two) {
    val counter = RegInit(0.U(32.W))

    counter := counter + 1.U

    io.count_two := counter
  }

}

verilogPrint(new MultiClockCounter)

In [ ]:
class TopLevel extends RawModule {
  val io = IO(new Bundle {
    val clk = Input(Clock())
    val reset = Input(AsyncReset())

    val dbg_clk = Input(Clock())
    val dbg_reset = Input(AsyncReset())

    val data = new QueueIO(UInt(32.W), 32)
    val dbg = new QueueIO(UInt(32.W), 32)

  })

  withClockAndReset(io.clk, io.reset) {
    val fifo = Module(new Queue(UInt(32.W), 32))
    
    fifo.io <> io.data
  }

  withClockAndReset(io.dbg_clk, io.dbg_reset) {
    val dbg_fifo = Module(new Queue(UInt(32.W), 32))
    dbg_fifo.io <> io.dbg
  }

}

visualize(() => new TopLevel)

## BlackBox

Позволяет использовать модули непосредственно из `verilog`

In [ ]:
class ResetSync extends BlackBox {
  val io = IO(new Bundle {
    val clk_i = Input(Clock())
    val rst_n_i = Input(Bool())
    val scan_mode_i = Input(Bool())
    val rst_n_o = Output(Bool())
  })
}

class ResetSyncWrap extends RawModule {
  val io = IO(new Bundle {
    val clk_i = Input(Clock())
    val rst_n_i = Input(Bool())
    val scan_mode_i = Input(Bool())
    val rst_n_o = Output(Bool())
  })

  val sync = Module(new ResetSync)

  io.clk_i <> sync.io.clk_i
  io.rst_n_i <> sync.io.rst_n_i
  io.scan_mode_i <> sync.io.scan_mode_i
  io.rst_n_o <> sync.io.rst_n_o
}

verilogPrint(new ResetSyncWrap)

In [ ]:
class ResetSync extends BlackBox(Map(
  "PARAM_1" -> 32,
  "PARAM_2" -> "DEFAULT",
  "PARAM_3" -> RawParam("8'hFF")
)) {
  val io = IO(new Bundle {
    val clk_i = Input(Clock())
    val rst_n_i = Input(Bool())
    val scan_mode_i = Input(Bool())
    val rst_n_o = Output(Bool())
  })
}

class ResetSyncWrap extends RawModule {
  val io = IO(new Bundle {
    val clk_i = Input(Clock())
    val rst_n_i = Input(Bool())
    val scan_mode_i = Input(Bool())
    val rst_n_o = Output(Bool())
  })

  val sync = Module(new ResetSync)

  io.clk_i <> sync.io.clk_i
  io.rst_n_i <> sync.io.rst_n_i
  io.scan_mode_i <> sync.io.scan_mode_i
  io.rst_n_o <> sync.io.rst_n_o
}

verilogPrint(new ResetSyncWrap)

In [ ]:
class mux_4to1 extends BlackBox with HasBlackBoxResource {
  val io = IO(new Bundle {
    val a = Input(Bool())
    val b = Input(Bool())
    val c = Input(Bool())
    val d = Input(Bool())
    val out = Output(Bool())
  })

  setResource("resources/mux_4to1.v")
}

In [ ]:
class reset_sync extends BlackBox with HasBlackBoxInline {
  val io = IO(new Bundle {
    val clk_i = Input(Clock())
    val rst_n_i = Input(Bool())
    val scan_mode_i = Input(Bool())
    val rst_n_o = Output(Bool())
  })

  setInline("reset_sync.v",
  s"""
    |module reset_sync #(
    |  parameter SYNC_STAGE    = 3,
    |  parameter [0:0] ACTIVE_LEVEL  = 0
    |)(
    |  input  wire clk_i,
    |  input  wire rst_n_i,
    |  input  wire scan_mode_i,
    |  output wire rst_n_o
    |);
    | // ... some verilog code here
    |endmodule
  """.stripMargin)
}

class ResetSyncWrap extends RawModule {
  val io = IO(new Bundle {
    val clk_i = Input(Clock())
    val rst_n_i = Input(Bool())
    val scan_mode_i = Input(Bool())
    val rst_n_o = Output(Bool())
  })

  val sync = Module(new reset_sync)

  io.clk_i <> sync.io.clk_i
  io.rst_n_i <> sync.io.rst_n_i
  io.scan_mode_i <> sync.io.scan_mode_i
  io.rst_n_o <> sync.io.rst_n_o
}

verilogPrint(new ResetSyncWrap)

## Chisel Testers

В chisel присутсвует встроеный фрэймворк для симуляции поведения rtl, поддерживаются два backend симулятора - treadle и verilator.

Набор функциональности относительно простой

`poke` - устанавливает значение сигнала
`peek` - возвращает значение сигнала
`step(n: Int)` - временной шаг симуляции, где `n` - количество тактов сигнала `clock`

В новой верси библиотеки `chiseltest` функционал значительно расширен, добавлена поддержка multi clock, работа с decoupled интерфейсами из коробки, также асинхронная симуляция на основе `threads` в scala.


In [ ]:
import treadle._

import chisel3.tester._
import chisel3.tester.internal._
import chisel3.tester.experimental.TestOptionBuilder._

import chisel3.tester.RawTester.test



In [ ]:
class PassthroughGenerator(width: Int) extends Module { 
  val io = IO(new Bundle {
    val in = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  io.out := io.in
}

val annotations = Seq(
//   VerilatorBackendAnnotation,
//   WriteVcdAnnotation,
  VerboseAnnotation,
  SymbolsToWatchAnnotation(Seq("io_out"))
)

test(new PassthroughGenerator(16), annotations) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.io.in.poke(1.U)
    c.io.out.expect(1.U)
    c.io.in.poke(2.U)
    c.io.out.expect(2.U)
}

In [ ]:
case class QueueModule[T <: Data](ioType: T, entries: Int) extends MultiIOModule {
  val in = IO(Flipped(Decoupled(ioType)))
  val out = IO(Decoupled(ioType))
  out <> Queue(in, entries)
}

val annotations = Seq(
//   VerilatorBackendAnnotation,
//   WriteVcdAnnotation,
//   VerboseAnnotation,
//   SymbolsToWatchAnnotation(Seq("out_valid"))
)

test(QueueModule(UInt(9.W), entries = 200), annotations) { c =>
    c.in.initSource()
    c.in.setSourceClock(c.clock)
    c.out.initSink()
    c.out.setSinkClock(c.clock)
    
    val testVector = Seq.tabulate(200){ i => i.U }

    testVector.zip(testVector).foreach { case (in, out) =>
      c.in.enqueueNow(in)
      c.out.expectDequeueNow(out)
    }
}

## Материалы

* https://github.com/freechipsproject/chisel3/wiki
* https://github.com/freechipsproject/chisel-bootcamp
* https://www.youtube.com/watch?v=2-ZiXNd9wbc
* https://www.youtube.com/watch?v=OhMuPQcyynY
* https://habr.com/ru/post/419413/
* API и Исходный код!